**Preprocesado del dataset**

*   Angel Luis Parra Marcos


## Dataset extraído de:
### https://www.kaggle.com/datasets/agungpambudi/property-sales-data-real-estate-trends?resource=download
### Contiene información de venta inmobiliarias en el estado de Milwaukee(USA) desde el 2002 hasta el 2022

## Carga de Datos

In [1]:
import pandas as pd
import numpy as np
import os
import plotnine as pn
from plotnine import *
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
propiedades_2018 = pd.read_csv('2002-2018-property-sales-data.csv')
propiedades_2019 = pd.read_csv('2019-property-sales-data.csv')
propiedades_2020 = pd.read_csv('2020-property-sales-data.csv')
propiedades_2021 = pd.read_csv('2021-property-sales-data.csv')
propiedades_2022 = pd.read_csv('2022-property-sales-data.csv')

## Preprocesamiento y limpieza


### Unificación columnas con misma información:
Ya que nuestros datos están divididos en diferentes archivos que no tienen una estructura homogenea, analizamos las diferencias de las variables e intentamos unificar la información.

 (Ej. 'Taxkey' y 'taxkey')

In [4]:
print(propiedades_2018.columns, propiedades_2019.columns, propiedades_2020.columns, propiedades_2021.columns, propiedades_2022.columns)

Index(['PropType', 'Taxkey', 'Address', 'CondoProject', 'District', 'Nbhd',
       'Style', 'Extwall', 'Stories', 'Year_Built', 'Nr_of_rms', 'Fin_sqft',
       'Units', 'Bdrms', 'Fbath', 'Hbath', 'Lotsize', 'Sale_date',
       'Sale_price'],
      dtype='object') Index(['PropertyID', 'PropType', 'taxkey', 'Address', 'CondoProject',
       'District', 'nbhd', 'Style', 'Extwall', 'Stories', 'Year_Built',
       'Rooms', 'FinishedSqft', 'Units', 'Bdrms', 'Fbath', 'Hbath', 'Lotsize',
       'Sale_date', 'Sale_price'],
      dtype='object') Index(['PropertyID', 'PropType', 'taxkey', 'Address', 'CondoProject',
       'District', 'nbhd', 'Style', 'Extwall', 'Stories', 'Year_Built',
       'Rooms', 'FinishedSqft', 'Units', 'Bdrms', 'Fbath', 'Hbath', 'Lotsize',
       'Sale_date', 'Sale_price'],
      dtype='object') Index(['PropertyID', 'PropType', 'taxkey', 'Address', 'CondoProject',
       'District', 'nbhd', 'Style', 'Extwall', 'Stories', 'Year_Built',
       'Rooms', 'FinishedSqft', 'Units

Se puede observar que en los años 2019-2022 hay una columna extra de property ID:
1. Pasar a minusculas todas las columnas
2. Eliminar la columna propertyid: Ya que no queremos que el modelo aprenda de identificadores únicos, además esta columna no aparece en los datos del 2002 al 2018

In [5]:
propiedades_2018.columns = propiedades_2018.columns.str.lower()
propiedades_2019.columns = propiedades_2019.columns.str.lower()
propiedades_2020.columns = propiedades_2020.columns.str.lower()
propiedades_2021.columns = propiedades_2021.columns.str.lower()
propiedades_2022.columns = propiedades_2022.columns.str.lower()
propiedades_2019 = propiedades_2019.drop(columns=['propertyid'], errors='ignore')
propiedades_2020 = propiedades_2020.drop(columns=['propertyid'], errors='ignore')
propiedades_2021 = propiedades_2021.drop(columns=['propertyid'], errors='ignore')
propiedades_2022 = propiedades_2022.drop(columns=['propertyid'], errors='ignore')

In [6]:
propiedades_2018.rename(columns={'nr_of_rms': 'rooms'}, inplace=True)
propiedades_2018.rename(columns={'fin_sqft': 'finishedsqft'}, inplace=True)
propiedades_2018.rename(columns={'fin_sqft': 'finishedsqft'}, inplace=True)

In [7]:
# Concatenamos
propiedades = pd.concat([propiedades_2018, propiedades_2019, propiedades_2020, propiedades_2021, propiedades_2022], ignore_index=True)

In [8]:
propiedades.shape[0]
# Hay un total de 60743 filas

60743

In [9]:
#Revisamos que no hayan columnas duplicadas
propiedades.columns

Index(['proptype', 'taxkey', 'address', 'condoproject', 'district', 'nbhd',
       'style', 'extwall', 'stories', 'year_built', 'rooms', 'finishedsqft',
       'units', 'bdrms', 'fbath', 'hbath', 'lotsize', 'sale_date',
       'sale_price'],
      dtype='object')

In [10]:
# Hacemos una inspeccion de las primeras filas
propiedades.head(5)

,proptype,taxkey,address,condoproject,district,nbhd,style,extwall,stories,year_built,rooms,finishedsqft,units,bdrms,fbath,hbath,lotsize,sale_date,sale_price
0,Commercial,3.230461e+09,2628 N 6TH ST,NaN,6.0,6258.0,Commercial Exempt,NaN,2.0,1880.0,0.0,1840,1.0,0.0,0.0,0.0,12750,2002-01,15900
1,Commercial,3.590192e+09,1363 N PROSPECT AV,NaN,3.0,6262.0,Mansions With Commercial Usage,NaN,2.0,1876.0,0.0,6377,1.0,0.0,0.0,0.0,11840,2002-01,850000
2,Commercial,4.161194e+09,617 S 94TH ST,NaN,10.0,6272.0,Service Building,NaN,1.0,1954.0,0.0,5022,1.0,0.0,0.0,0.0,9700,2002-01,119000
3,Commercial,1.719836e+09,3624 W SILVER SPRING DR,NaN,1.0,6218.0,"Store Bldg - Multi Story (Store & Apt, Store & O",NaN,2.0,1955.0,0.0,6420,1.0,0.0,0.0,0.0,8792,2002-01,210000
4,Commercial,3.480290e+09,3830 W LISBON AV,NaN,15.0,6254.0,"Store Bldg - Multi Story (Store & Apt, Store & O",NaN,2.0,1909.0,0.0,5956,1.0,0.0,0.0,0.0,4840,2002-01,48500


In [11]:
propiedades_2022

,proptype,taxkey,address,condoproject,district,nbhd,style,extwall,stories,year_built,rooms,finishedsqft,units,bdrms,fbath,hbath,lotsize,sale_date,sale_price
0,Manufacturing,30131000,9434-9446 N 107TH ST,NaN,9,6300,Service Building,Concrete Block,1.0,1978.0,NaN,20600.0,6,NaN,0,0,0,2022-04-01,950000.0
1,Commercial,30152000,9306-9316 N 107TH ST,NaN,9,6202,Office Building - 1 Story,Brick,1.0,1982.0,NaN,9688.0,23,NaN,0,0,35719,2022-10-07,385000.0
2,Residential,49980110,9327 N SWAN RD,NaN,9,40,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,1341648,2022-01-07,800000.0
3,Residential,49993200,9411 W COUNTY LINE RD,NaN,9,40,Ranch,Aluminum/Vinyl,1.0,1959.0,6.0,1334.0,1,3.0,1,1,83200,2022-08-09,280000.0
4,Residential,50042000,9322 N JOYCE AV,NaN,9,40,Ranch,Aluminum/Vinyl,1.0,1980.0,10.0,1006.0,1,6.0,1,0,8303,2022-05-23,233100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7141,Condominium,7160339000,1912 W SALEM ST,COLLEGE HEIGHTS,13,5360,Low Rise 1-3 Stories,NaN,2.0,1974.0,10.0,1100.0,1,4.0,1,1,1,2022-03-11,125900.0
7142,Condominium,7160351000,6316 S 20TH ST,COLLEGE HEIGHTS,13,5360,Low Rise 1-3 Stories,NaN,1.0,1974.0,5.0,1379.0,1,2.0,1,1,1,2022-10-28,150000.0
7143,Condominium,7160365000,6376 S 20TH ST,COLLEGE HEIGHTS,13,5360,Low Rise 1-3 Stories,NaN,2.0,1974.0,10.0,1100.0,1,4.0,1,1,1,2022-03-15,130000.0
7144,Condominium,7160366000,6378 S 20TH ST,COLLEGE HEIGHTS,13,5360,Low Rise 1-3 Stories,NaN,2.0,1974.0,5.0,1100.0,1,2.0,1,1,1,2022-12-30,123000.0


## Revisión y corrección de tipos
A Continuación presentamos la información de la fuente del dataset acerca de la descripcion de variables y tipos

| Field Name       | Description                                                   | Type   |
|------------------|---------------------------------------------------------------|--------|
| PropertyID       | A unique identifier for each property.                        | text   |
| PropType         | The type of property (e.g., Commercial or Residential).       | text   |
| taxkey           | The tax key associated with the property.                     | text   |
| Address          | The address of the property.                                  | text   |
| CondoProject     | Information about whether the property is part of a condominium project (NaN indicates missing data). | text |
| District         | The district number for the property.                         | text   |
| nbhd             | The neighborhood number for the property.                     | text   |
| Style            | The architectural style of the property.                      | text   |
| Extwall          | The type of exterior wall material used.                      | text   |
| Stories          | The number of stories in the building.                        | text   |
| Year_Built       | The year the property was built.                              | text   |
| Rooms            | The number of rooms in the property.                          | text   |
| FinishedSqft     | The total square footage of finished space in the property.   | text   |
| Units            | The number of units in the property (e.g., apartments in a multifamily building). | text |
| Bdrms            | The number of bedrooms in the property.                       | text   |
| Fbath            | The number of full bathrooms in the property.                 | text   |
| Hbath            | The number of half bathrooms in the property.                 | text   |
| Lotsize          | The size of the lot associated with the property.              | text   |
| Sale_date        | The date when the property was sold.                           | text   |
| Sale_price       | The sale price of the property.                               | text   |

Como podemos ver no es muy especifico acerca de los tipos a usar y la descripción de el contenido es muy generica en algunas columnas, en base a esto haremos la definición de los tipos

In [12]:
propiedades.dtypes

proptype         object
taxkey          float64
address          object
condoproject     object
district        float64
nbhd            float64
style            object
extwall          object
stories         float64
year_built      float64
rooms           float64
finishedsqft     object
units           float64
bdrms           float64
fbath           float64
hbath           float64
lotsize          object
sale_date        object
sale_price       object
dtype: object

### Sale_date:
Se evidencian diferentes formatos de fechas en los diferentes archivos.

In [13]:
print(propiedades_2018['sale_date'][0])
print(propiedades_2019['sale_date'][0])
print(propiedades_2020['sale_date'][0])
print(propiedades_2021['sale_date'][0])
print(propiedades_2022['sale_date'][0])

2002-01
3/11/2019
2/13/2020
3/31/2021
2022-04-01


In [14]:
## Transformamos a formato fecha , usando la funcion de pandas que, nos permite inferir varios formatos y posteriormente lo pasamos a un formato unico
propiedades['sale_date'] = pd.to_datetime(propiedades['sale_date'], format='mixed')
propiedades['sale_date'] = pd.to_datetime(propiedades['sale_date'], format='%m/%d/%Y')


In [17]:
#CAMBIO DE TIPOS DE LAS VARIABLES
propiedades['proptype'] = propiedades['proptype'].astype('category')
propiedades['district'] = propiedades['district'].astype('category')
propiedades['stories'] = propiedades['stories'].astype('category')
propiedades['extwall'] = propiedades['extwall'].astype('category')
propiedades['rooms'] = propiedades['rooms'].astype('category')
propiedades['units'] = propiedades['units'].astype('category')
propiedades['bdrms'] = propiedades['bdrms'].astype('category')
propiedades['fbath'] = propiedades['fbath'].astype('category')
propiedades['hbath'] = propiedades['hbath'].astype('category')
propiedades['taxkey'] = propiedades['taxkey'].astype('category')

In [18]:
# No se puede pasar Sale_price directamente a numeric, ya que hay valores que contienen $, primero remplazar y luego convertir
propiedades['sale_price'] = propiedades['sale_price'].replace('[\$]', '', regex=True)
propiedades['sale_price'] = propiedades['sale_price'].replace(',', '', regex=True)
propiedades['sale_price'] = propiedades['sale_price'].astype(float)

In [19]:
propiedades['nbhd'] =propiedades['nbhd'].astype('category')

In [20]:
propiedades['finishedsqft'] = propiedades['finishedsqft'].astype(str)
propiedades['finishedsqft'] = pd.to_numeric(propiedades['finishedsqft'].str.replace(',', ''), errors='coerce')


In [21]:
propiedades.dtypes

proptype              category
taxkey                category
address                 object
condoproject            object
district              category
nbhd                  category
style                   object
extwall               category
stories               category
year_built             float64
rooms                 category
finishedsqft           float64
units                 category
bdrms                 category
fbath                 category
hbath                 category
lotsize                 object
sale_date       datetime64[ns]
sale_price             float64
dtype: object

In [23]:
propiedades['lotsize'] = propiedades['lotsize'].astype(str)
propiedades['lotsize'] = propiedades['lotsize'].str.replace(',', '')


In [24]:
propiedades.isnull().sum()

proptype           13
taxkey              2
address             6
condoproject    50662
district            4
nbhd               59
style              76
extwall         13471
stories           131
year_built         44
rooms            1273
finishedsqft       77
units               2
bdrms            1272
fbath             657
hbath            4842
lotsize             0
sale_date           2
sale_price         12
dtype: int64

In [25]:
propiedades.shape[0]

60743

### year_built

In [26]:
propiedades['year_built'].unique()

array([1880., 1876., 1954., 1955., 1909., 1974., 2002., 1887., 1890.,
       1913.,    0., 1981., 1905., 1929., 1993., 1957., 1885., 1953.,
       1870., 1969., 1958., 1985., 1947., 1964., 1948., 1915., 1928.,
       1924., 1990., 1989., 1946., 1941., 1926., 1923., 1984., 1949.,
       1912., 1972., 1895., 1897., 1980., 1918., 1914., 1936., 1965.,
       1907., 1911., 1927., 1961., 1975., 1967., 1950., 1973., 1920.,
       1956., 1968., 1982., 1933., 1935., 1991., 1988., 1963., 1910.,
       1925., 1995., 1986., 1922., 1952., 1945., 1916., 1891., 1906.,
       1902., 1930., 1898., 1977., 1996., 1951., 1934., 1970., 1962.,
       1900., 1896., 1921., 2001., 1966., 1868., 1901., 1889., 2000.,
       1931., 1882., 1987., 1959., 1976., 2003., 1904., 1960., 1903.,
       2005., 1994., 1940., 1888., 1884., 1937., 1938., 1899., 1979.,
       1932., 2006., 2004., 1893., 1892., 1998., 1860., 1992., 1997.,
       1971., 1917., 1894., 1978., 1919., 1908., 1865., 1983., 1944.,
       2008., 2007.,

Hay 638 observaciones de los años 206, 203 y 0 que estan completamente vacios.
Si los elimino paso de tener 60.743 a 60.105
A parte hay 44 valores de año vacios
pasando a tener 60.061


In [27]:
num_anomalias_year = propiedades['year_built'].isin([206, 203, 0]).sum()
num_missings_year = propiedades['year_built'].isnull().sum()
print( 'Registros anomalos en año de construcción',num_anomalias_year + num_missings_year)

Registros anomalos en año de construcción 682


Considerando que el año de construcción es una variable dificilmente imputable, decidimos eliminar estas filas

In [28]:
# Estos tres años solo contienen valores nulos y son clarosn valores atipicos en esta variable por lo que decidimos eliminarlos
propiedades = propiedades.dropna(subset=['year_built'])
propiedades = propiedades[~propiedades['year_built'].isin([206, 203, 0])]
propiedades.shape[0]

60061

### Remoción de filas duplicadas


99 duplicados

In [29]:
propiedades.duplicated().sum()

99

In [30]:
propiedades = propiedades.drop_duplicates()
propiedades.duplicated().sum()

0

In [31]:
propiedades.shape[0]

59962

### Busqueda y resolución NA



In [32]:
propiedades.isnull().sum()

proptype            9
taxkey              0
address             4
condoproject    49988
district            2
nbhd               56
style               6
extwall         12775
stories            90
year_built          0
rooms            1156
finishedsqft        0
units               0
bdrms            1155
fbath             586
hbath            4769
lotsize             0
sale_date           0
sale_price         10
dtype: int64

Hacemos un analisis de los missing values
Las columnas que mas nulls presentan son
1. condoproject: decidimos eliminarla ya que es en su mayoria missing
2. extwall: decidimos eliminarla ya que casi 1/4 del dataset es missing
3. rooms
4. bdrms
5. hbath

In [33]:
propiedades = propiedades.drop(['condoproject'], axis=1)

In [34]:
propiedades = propiedades.drop(['extwall'], axis=1)

In [35]:
propiedades.isnull().sum()

proptype           9
taxkey             0
address            4
district           2
nbhd              56
style              6
stories           90
year_built         0
rooms           1156
finishedsqft       0
units              0
bdrms           1155
fbath            586
hbath           4769
lotsize            0
sale_date          0
sale_price        10
dtype: int64

In [36]:
propiedades.shape[0]

59962

Ademas decidimos eliminar algunos missings de las variables importantes y target con pocos missings.


In [37]:
propiedades = propiedades.dropna(subset=['proptype', 'style', 'address', 'district', 'sale_price'])

In [38]:
propiedades.isnull().sum()

proptype           0
taxkey             0
address            0
district           0
nbhd              56
style              0
stories           90
year_built         0
rooms           1152
finishedsqft       0
units              0
bdrms           1151
fbath            583
hbath           4754
lotsize            0
sale_date          0
sale_price         0
dtype: int64

### Imputación, Limpieza y creación columnas auxiliares

### proptype:
Vemos que hay 7 categorias

In [39]:
propiedades['proptype'].unique()

['Commercial', 'Residential', 'Lg Apartment', 'Condominium', 'Vacant Land', 'Manufacturing', 'Exempt']
Categories (7, object): ['Commercial', 'Condominium', 'Exempt', 'Lg Apartment', 'Manufacturing', 'Residential', 'Vacant Land']

In [40]:
propiedades['proptype'].value_counts().head(25)

proptype
Residential      45506
Condominium       9041
Commercial        3463
Lg Apartment      1896
Manufacturing       14
Vacant Land          7
Exempt               4
Name: count, dtype: int64

### Style

Dentro de la columna style vemos que hay muchisimas categorias, pero que contienen muchas la misma informacion, ejemplo:

- Apartment: 24: 2-Br ,3: 3-Br  Units
- Apartment: 81: 1-Br  Units

Trataremos de extraer esta información


In [41]:
len(propiedades)

59931

In [42]:
propiedades['style'].unique()

array(['Commercial Exempt', 'Mansions With Commercial Usage',
       'Service Building',
       'Store Bldg - Multi Story (Store & Apt, Store & O',
       'Strip Shopping Center', 'Svs Station w Conv Store', 'Tavern',
       'AP 1', 'Day Care Center', 'Duplex Old/Style',
       'Office Building - Multi Story ( Ofc & Apt)', 'Pole Building',
       'Residence With Commercial Usage', 'Used Car Sale',
       'Warehouse Building - 1 Story', 'Apartment: 48: 1-Br  Units',
       'Beer Depot & Liquor Store',
       'Store Building - Single tenant, 1 story', 'Fast Food Operation',
       'Multi Story Warehouse', 'Sit Down Restaurant', 'Triplex',
       'Rm or Rooming House', 'Apartment: 32: 2-Br  Units',
       'Laundry Pick-up Station', 'Auto & Auto Body Repair',
       'Custard Stand, Drive-In', 'Muffler Shop', 'Salvage Yard',
       'Super Market', 'Office Building - 1 Story',
       'Office Building - Class A', 'Svc Station w Conv and Carwash',
       'Apartment: 37: 1-Br ,1: 2-Br  Units', 

In [43]:
propiedades['style'] = propiedades['style'].str.lower()
propiedades['style'] = propiedades['style'].str.split(': | -').str[0].str.strip()
propiedades['style'].value_counts().head(10)

style
ranch                    13140
cape cod                  8887
apartment                 4603
milwaukee bungalow        3667
duplex o/s                3391
residence o/s             3303
colonial                  2731
dplx bungalow             2662
duplex n/s                2348
mid rise 4-12 stories     1356
Name: count, dtype: int64

Creado una nueva columna con datos mas limpios, quedandonos con la informacion especifica de estilo de propiedad necesaria para poder hacer graficos

In [44]:
propiedades[propiedades['rooms'].isnull()]

,proptype,taxkey,address,district,nbhd,style,stories,year_built,rooms,finishedsqft,units,bdrms,fbath,hbath,lotsize,sale_date,sale_price
34523,Manufacturing,1.001100e+07,9400 N 124TH ST,9.0,6300.0,pole building,1.0,2005.0,NaN,18000.0,1.0,NaN,NaN,NaN,0.0,2019-03-11,675000.0
34524,Commercial,3.002311e+07,10545 W DONGES CT,9.0,6202.0,warehouse building,1.0,1973.0,NaN,27940.0,5.0,NaN,NaN,NaN,100188.0,2019-09-13,1300000.0
34547,Commercial,3.300151e+08,8530-8564 W BROWN DEER RD,9.0,6452.0,strip shopping center,1.0,1984.0,NaN,95932.0,13.0,NaN,NaN,NaN,0.0,2019-04-22,725000.0
34627,Commercial,4.299981e+08,8730 N 91ST ST,9.0,6421.0,"car dealer, truck dealer",1.0,1992.0,NaN,14165.0,2.0,NaN,NaN,NaN,187308.0,2019-10-03,850000.0
34652,Commercial,4.308710e+08,8155-8173 W BROWN DEER RD,9.0,6452.0,neighborhood center,1.0,1975.0,NaN,141851.0,22.0,NaN,NaN,NaN,0.0,2019-01-09,1150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60642,Lg Apartment,6.680209e+09,3047 W IONA TRCE,13.0,6850.0,ap1 (conventional apt with 4-6 units),2.0,1959.0,NaN,3258.0,4.0,NaN,0.0,0.0,8710,2022-07-01,357142.0
60643,Lg Apartment,6.680231e+09,3016 W IONA TRCE,13.0,6850.0,ap1 (conventional apt with 4-6 units),2.0,1959.0,NaN,3696.0,4.0,NaN,0.0,0.0,8970,2022-08-31,325000.0
60644,Commercial,6.680702e+09,2730 W RAMSEY AV,13.0,6449.0,nursing home,1.0,1970.0,NaN,62714.0,81.0,NaN,0.0,0.0,142397,2022-10-13,3366096.0
60682,Residential,6.709944e+09,1806 W RAMSEY AV,13.0,4780.0,cape cod,1.5,1939.0,NaN,1900.0,1.0,NaN,1.0,1.0,21519,2022-06-23,310000.0


In [45]:
upper_limit = np.percentile(propiedades['sale_price'], 95)
lower_limit = np.percentile(propiedades['sale_price'], 5)

propiedades['sale_price'] = np.where(propiedades['sale_price'] > upper_limit, upper_limit,
                                            np.where(propiedades['sale_price'] < lower_limit, lower_limit,
                                                     propiedades['sale_price']))


In [46]:
propiedades.dtypes
propiedades['lotsize'] =propiedades['lotsize'].astype('float64')

In [47]:
col_otliers_clean = [ 'finishedsqft', 'lotsize',]
for col in col_otliers_clean:
    # Define percentiles for capping
    upper_limit = np.percentile(propiedades[col], 95)
    lower_limit = np.percentile(propiedades[col], 5)

    # Cap outliers
    propiedades[col] = np.where(propiedades[col] > upper_limit, upper_limit,
                                            np.where(propiedades[col] < lower_limit, lower_limit,
                                                     propiedades[col]))


In [48]:
columns_to_impute = ['nbhd', 'rooms', 'bdrms', 'fbath', 'hbath', 'stories']


propiedades[columns_to_impute] = propiedades[columns_to_impute].astype(float)

propiedades.loc[:, columns_to_impute] = propiedades[columns_to_impute].fillna(0)


propiedades[columns_to_impute] = propiedades[columns_to_impute].astype(int)

In [49]:
propiedades.head()
propiedades.to_csv('propiedades_clean.csv', index=False)

In [50]:
propiedades.isnull().sum()

proptype        0
taxkey          0
address         0
district        0
nbhd            0
style           0
stories         0
year_built      0
rooms           0
finishedsqft    0
units           0
bdrms           0
fbath           0
hbath           0
lotsize         1
sale_date       0
sale_price      0
dtype: int64